In [1]:
# Author : Arthur Prigent
# Email: aprigent@geomar.de

In [2]:
from load_librairies import *
import xscale.signal.fitting as xsf

dir_hist_sst = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/historical/tos/mon/'
dir_ssp585_sst = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/ssp585/tos/mon/'
dir_ssp126_sst = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/ssp126/tos/mon/'
dir_ssp245_sst = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/ssp245/tos/mon/'
dir_ssp370_sst = '/Volumes/Arthur_disk2/Data/CMIP6/Omon/ssp370/tos/mon/'
path_fig = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/figures/'
lon_min = -20
lon_max = 0
lat_min = -3
lat_max = 3

period_str_1 = 1950
period_end_1 = 1999

period_str_2 = 2050
period_end_2 = 2099

def nandetrend(y):
    ''' Remove the linear trend from the data '''
    x = np.arange(0,y.shape[0],1)
    
    not_nan_ind = ~np.isnan(y)
    m, b, r_val, p_val, std_err = stats.linregress(x[not_nan_ind],np.array(y)[not_nan_ind])
    
    
    y_detrended_tmp= np.array(y) - m*x
    
    y_detrended  = xr.Dataset({'detrended': (['time'],y_detrended_tmp)}
                       ,coords={'time':y.time[:]})
    return y_detrended

def is_jja(month):
    return (month >= 6) & (month <= 8)
now = datetime.now()
date_time = now.strftime("%d/%m/%Y")

path_data = '/Users/aprigent/Documents/Thesis_GEOMAR/Projects/weakened_sst_variability_CMIP5/scripts/new_version/to_publish/data/'

In [3]:
model = ['ACCESS-CM2',
         'ACCESS-ESM1-5', 'BCC-CSM2-MR',
         'CAMS-CSM1-0', 'CanESM5', 'EC-Earth3', 'EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
         'INM-CM4-8', 'INM-CM5-0',
         'IPSL-CM6A-LR', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0','NESM3']


In [4]:
sst_mean_model_jja_1 = np.ones((len(model),60,81))*np.nan
sst_mean_model_jja_2 = np.ones((len(model),60,81))*np.nan


sst_mean_model_1 = np.ones((len(model),60,81))*np.nan
sst_mean_model_2 = np.ones((len(model),60,81))*np.nan


for i in range(len(model)):
    try:
        nc_tmp = xr.open_dataset(dir_hist_sst+'tos_Omon_'+model[i]+'_historical_r1i1p1f1_187001-201412_1deg.nc')
        print(model[i])
        sst_tmp_1 = nc_tmp.tos[:]

        try:

            sst_tmp_1 = sst_tmp_1.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))
        except TypeError:

            sst_tmp_1['time'] = sst_tmp_1.indexes['time'].to_datetimeindex()
            sst_tmp_1 = sst_tmp_1.sel(time=slice(datetime(period_str_1, 1, 1), datetime(period_end_1, 12, 31)))

        sst_tmp_1 = xr.concat([sst_tmp_1[ :,:, 180:], sst_tmp_1[:, :, :180]], dim='lon')
        sst_tmp_1.coords['lon'] = (sst_tmp_1.coords['lon'] + 180) % 360 - 180
        sst_tmp_1 = Atools.data_sub(sst_tmp_1,-60,20,-30,30)
        
        sst_jja_mean = sst_tmp_1.sel(time=is_jja(sst_tmp_1['time.month'])).mean(dim='time')
        sst_mean = sst_tmp_1.mean(dim='time')

        sst_mean_model_jja_1[i,:,:] = sst_jja_mean.values
        sst_mean_model_1[i,:,:] = sst_mean.values
    except FileNotFoundError:
        print('No file, pass')
        sst_mean_model_jja_1[i,:,:] =np.ones((60,81))*np.nan
        sst_mean_model_1[i,:,:] = np.ones((60,81))*np.nan
    
    ## SSP595 ##
    try:
        nc_tmp = xr.open_dataset(dir_ssp585_sst+'tos_Omon_'+model[i]+'_ssp585_r1i1p1f1_201501-209912_1deg.nc')
        print(model[i])
        sst_tmp_2 = nc_tmp.tos[:]
        try:

            sst_tmp_2 = sst_tmp_2.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))
        except TypeError:

            sst_tmp_2['time'] = sst_tmp_2.indexes['time'].to_datetimeindex()
            sst_tmp_2 = sst_tmp_2.sel(time=slice(datetime(period_str_2, 1, 1), datetime(period_end_2, 12, 31)))

        sst_tmp_2 = xr.concat([sst_tmp_2[ :,:, 180:], sst_tmp_2[:, :, :180]], dim='lon')
        sst_tmp_2.coords['lon'] = (sst_tmp_2.coords['lon'] + 180) % 360 - 180
        sst_tmp_2 = Atools.data_sub(sst_tmp_2,-60,20,-30,30)
        
        sst_jja_mean = sst_tmp_2.sel(time=is_jja(sst_tmp_2['time.month'])).mean(dim='time')
        sst_mean = sst_tmp_2.mean(dim='time')

        sst_mean_model_jja_2[i,:,:] = sst_jja_mean.values
        sst_mean_model_2[i,:,:] = sst_mean.values
    except FileNotFoundError:
        print('No file, pass')
        sst_mean_model_jja_2[i] =np.ones((60,81))*np.nan
        sst_mean_model_2[i] = np.ones((60,81))*np.nan
    
   

ACCESS-CM2
ACCESS-CM2
ACCESS-ESM1-5
ACCESS-ESM1-5
BCC-CSM2-MR


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


BCC-CSM2-MR


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


CAMS-CSM1-0


...

/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


IPSL-CM6A-LR
IPSL-CM6A-LR
MIROC6
MIROC6
MPI-ESM1-2-HR
MPI-ESM1-2-HR
MPI-ESM1-2-LR
MPI-ESM1-2-LR
MRI-ESM2-0
MRI-ESM2-0
NESM3


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


NESM3


/Users/aprigent/miniconda3/envs/py3_std/lib/python3.7/site-packages/ipykernel_launcher.py:47: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


In [5]:

sst_mean_model_1_dataset  = xr.Dataset({'jja_sst': (['model','lat','lon'],sst_mean_model_jja_1),
                                       'sst': (['model','lat','lon'],sst_mean_model_1)}
                           ,coords={'model': model,
                                    'lon':np.array(sst_tmp_1.lon),
                                   'lat':np.array(sst_tmp_1.lat)}

                           ,attrs={'standard_name': 'sst',
                                    'long_name': 'Mean SST',
                                    'units': 'K',
                                    'model': 'CMIP6',
                                    'Scenario': 'Historical',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})


sst_mean_model_2_dataset  = xr.Dataset({'jja_sst': (['model','lat','lon'],sst_mean_model_jja_2),
                                       'sst': (['model','lat','lon'],sst_mean_model_2)}
                           ,coords={'model': model,
                                    'lon':np.array(sst_tmp_2.lon),
                                   'lat':np.array(sst_tmp_2.lat)}

                           ,attrs={'standard_name': 'sst',
                                    'long_name': 'Mean SST',
                                    'units': 'K',
                                    'model': 'CMIP6',
                                    'Scenario': 'SSP585',
                                    'Creation_date':date_time,   
                                    'author': 'Arthur Prigent'})



In [6]:
sst_mean_model_1_dataset.to_netcdf(path_data+'sst_mean_atl_JJA_diff_cmip6_hist.nc')
sst_mean_model_2_dataset.to_netcdf(path_data+'sst_mean_atl_JJA_diff_cmip6_rcp85.nc')
